<a href="https://colab.research.google.com/github/skhazaei/TensorFlow-repo/blob/master/diabetes_dataset_regularisation_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and return the diabetes dataset (regression) from `sklearn.datasets`


In [1]:
from sklearn.datasets import load_diabetes
diabete_dataset = load_diabetes()

In [2]:
print(diabete_dataset.keys())

dict_keys(['data', 'target', 'DESCR', 'feature_names', 'data_filename', 'target_filename'])


In [3]:
data = diabete_dataset['data']
target = diabete_dataset['target']

In [4]:
target = (target - target.mean(axis=0)) / target.std()

In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.1)

In [6]:
print(f'train data shape: {train_data.shape}')
print(f'train target shape: {train_target.shape}')
print(f'test data shape: {test_data.shape}')
print(f'test target shape: {test_target.shape}')

train data shape: (397, 10)
train target shape: (397,)
test data shape: (45, 10)
test target shape: (45,)


In [7]:
import tensorflow as tf

def get_regularized_model(wd, rate):
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(128, activation='relu', input_shape=(train_data.shape[1],),
                                                            kernel_regularizer=tf.keras.regularizers.l2(wd)),
                                      tf.keras.layers.Dropout(rate),
                                      tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(rate)),
                                      tf.keras.layers.Dropout(rate),
                                      tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(rate)),
                                      tf.keras.layers.Dropout(rate),
                                      tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(rate)),
                                      tf.keras.layers.Dropout(rate),
                                      tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(rate)),
                                      tf.keras.layers.Dropout(rate),
                                      tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(rate)),
                                      tf.keras.layers.Dropout(rate),
                                      tf.keras.layers.Dense(1)
                                      ])
  return model


# Define a training callback

In [8]:
class myTrainingCallback(tf.keras.callbacks.Callback):
  
  def on_train_begin(self, logs=None):
    print(f'Starting training ...')

  def on_epoch_begin(self, epoch, logs=None):
    print(f'Starting epoch: {epoch}')

  def on_train_batch_begin(self, batch, logs=None):
    print(f'Training: starting batch {batch}')
  
  def on_train_batch_end(self, batch, logs=None):
    print(f'Training: finished batch {batch}')
  
  def on_epoch_end(self, epoch, logs=None):
    print(f'Finished epoch: {epoch}')

  def on_train_end(self, logs=None):
    print(f'Finished training.')

In [9]:
model = get_regularized_model(1e-5, 0.3)

In [10]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the model with the callback

In [11]:
history = model.fit(train_data, train_target, validation_split=0.15,
                    epochs= 4, batch_size=64, callbacks=[myTrainingCallback()], verbose=2)

Starting training ...
Epoch 1/4
Starting epoch: 0
Training: starting batch 0
Training: finished batch 0
Training: starting batch 1
Training: finished batch 1
Training: starting batch 2
Training: finished batch 2
Training: starting batch 3
Training: finished batch 3
Training: starting batch 4
Training: finished batch 4
Training: starting batch 5
Training: finished batch 5
6/6 - 2s - loss: 184.3621 - accuracy: 0.0000e+00 - val_loss: 170.7771 - val_accuracy: 0.0000e+00
Finished epoch: 0
Epoch 2/4
Starting epoch: 1
Training: starting batch 0
Training: finished batch 0
Training: starting batch 1
Training: finished batch 1
Training: starting batch 2
Training: finished batch 2
Training: starting batch 3
Training: finished batch 3
Training: starting batch 4
Training: finished batch 4
Training: starting batch 5
Training: finished batch 5
6/6 - 0s - loss: 163.4277 - accuracy: 0.0000e+00 - val_loss: 151.0314 - val_accuracy: 0.0000e+00
Finished epoch: 1
Epoch 3/4
Starting epoch: 2
Training: starti

# Evaluate the model on test sets with callback

In [12]:
class myTestingCallback(tf.keras.callbacks.Callback):

  def on_test_begin(self, logs=None):
    print(f'Start testing')

  def on_epoch_begin(self, epoch, logs=None):
    print(f'Starting epoch {epoch}')

  def on_test_batch_begin(self, batch, logs=None):
    print(f'testing: starting batch {batch}')
  
  def on_test_batch_end(self, batch, logs=None):
    print(f'testing: finished batch {batch}')
  
  def on_epoch_end(self, epoch, logs=None):
    print(f'Finished epoch {epoch}')

  def on_test_end(self, logs=None):
    print(f'Finished testing')

In [13]:
model.evaluate(test_data, test_target, verbose=2, callbacks=[myTestingCallback()])

Start testing
testing: starting batch 0
testing: finished batch 0
testing: starting batch 1
testing: finished batch 1
2/2 - 0s - loss: 117.1653 - accuracy: 0.0000e+00
Finished testing


[117.16534423828125, 0.0]

# Prediction callback

In [14]:
class myPredictionCallback(tf.keras.callbacks.Callback):

  def on_predict_begin(self, logs=None):
    print(f'Starting prediction ...')

  def on_epoch_begin(self, epoch, logs=None):
    print(f'Prediction: epoch {epoch}')

  def on_predict_batch_begin(self, batch, logs=None):
    print(f'starting batch {batch}')

  def on_predict_batch_end(self, batch, logs=None):
    print(f'Finished batch {batch}')
  
  def on_epoch_end(self, epoch, logs=None):
    print(f'Finished epoch {epoch}')
  
  def on_predict_end(self, logs=None):
    print(f'Finished prediction')

In [15]:
model.predict(test_data, verbose=2, callbacks=[myPredictionCallback()])

Starting prediction ...
starting batch 0
Finished batch 0
starting batch 1
Finished batch 1
2/2 - 0s
Finished prediction


array([[-0.00452242],
       [-0.01416905],
       [-0.01085173],
       [-0.00522856],
       [-0.00982116],
       [ 0.00387532],
       [-0.00668642],
       [ 0.00713858],
       [-0.01257619],
       [-0.01102835],
       [ 0.00503015],
       [-0.01023284],
       [ 0.00084649],
       [ 0.00199564],
       [ 0.00476592],
       [ 0.00658903],
       [-0.00291959],
       [-0.01010456],
       [-0.0086866 ],
       [-0.0117946 ],
       [ 0.00189638],
       [-0.00825729],
       [-0.00924252],
       [ 0.00275086],
       [ 0.00129047],
       [-0.00809698],
       [ 0.00429168],
       [-0.00696066],
       [-0.00256508],
       [-0.00397631],
       [-0.00617415],
       [-0.0102554 ],
       [-0.00052834],
       [-0.00618516],
       [-0.01049056],
       [ 0.0002201 ],
       [ 0.00247102],
       [-0.00315312],
       [-0.00947857],
       [-0.00538108],
       [-0.01140872],
       [-0.00084271],
       [-0.00923288],
       [-0.00921156],
       [-0.0017106 ]], dtype=flo